### login client

In [ ]:
from binance.client import Client

In [ ]:
api_key = 'your_key';
api_secret = 'your_secret'

In [ ]:
client = Client(api_key, api_secret)

### get all orders; get order; get_open_orders; get_asset_balance; etc
note that market order doesn't show the price

In [ ]:
#?Client.get_all_orders

In [ ]:
#orders = client.get_all_orders(symbol='NEOUSDT')

In [ ]:
#order = client.get_order(symbol='NEOUSDT', orderId=7841515)
#print order['price']

In [ ]:
#for j in xrange(len(orders)-1, len(orders)-10, -1):
#    print orders[j]['status'], "   ", orders[j]['price'], "  ", orders[j]['side'], "  ", orders[j]['orderId']

In [ ]:
# order = client.get_open_orders(symbol="NEOUSDT")
# order = client.get_asset_balance(asset="BNB")
# print order

### place and cancel an order

#### create BUY and SELL limit orders  of NEO/USDT, quantity is 1, with price 146:

In [ ]:
order = client.create_order(
    symbol='NEOUSDT',
    side='BUY',
    type='LIMIT',
    timeInForce='GTC',
    quantity=1,
    price='146')


In [ ]:
order = client.create_order(
    symbol='NEOUSDT',
    side='SELL',
    type='LIMIT',
    timeInForce='GTC',
    quantity=1,
    price='146')

In [ ]:
print order

### cancel order

In [ ]:
result = client.cancel_order(
    symbol='NEOUSDT',
    orderId=order['orderId'])

In [ ]:
print result

In [ ]:
info = client.get_symbol_info('BNBBTC')

In [ ]:
import json
print json.dumps(info, indent=1)

In [ ]:
# depth = client.get_order_book(symbol='BNBBTC')
# print json.dumps(depth, indent=1)
# print depth['bids']

In [ ]:
# ?client.create_order

In [ ]:
# candles = client.get_klines(symbol='NEOUSDT', interval=Client.KLINE_INTERVAL_30MINUTE)

In [ ]:
# print json.dumps(candles[-2], indent=1)

In [ ]:
# ?Client.get_klines

In [ ]:
# ?client.get_ticker()

### make a limit order (BUY or SELL), based on current bid/ask queue. The following script tries to be first of the queue. (needs further coding)

In [3]:

def binanceorder(buyorsell, nums, client):
    '''
    buyorsell == 'b': BUY; else: SELL
    nums: how many quantity you want to trade (the following example only shows NEO/USDT case)
    '''
    price = client.get_ticker(symbol="NEOUSDT")
    gap = float(price['askPrice']) - float(price['bidPrice'])
    if buyorsell == 'b':
        myprice = float(price['bidPrice']) + gap * 0.01
        order = client.create_order(
            symbol='NEOUSDT',
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=nums,
            price=str(float("{0:.3f}".format(myprice))))
    else:
        myprice =  float(price['askPrice']) - gap*0.1
        order = client.create_order(
            symbol='NEOUSDT',
            side='SELL',
            type='LIMIT',
            timeInForce='GTC',
            quantity=nums,
            price=str(float("{0:.3f}".format(myprice))))
    return order


import time
def binanceorder_fight(buyorsell, nums, client):
    '''
    If the order is FILLED, good! If not, fight for the first place, with 5 sec frequency
    buyorsell == 'b': BUY; else: SELL
    nums: how many quantity you want to trade (the following example only shows NEO/USDT case)
    '''
    myorder = binanceorder(buyorsell, nums, client)
    time.sleep(0.1)
    currentorder = client.get_order(symbol='NEOUSDT', orderId=myorder['orderId'])
    while currentorder['status']!='FILLED': # if filed, done!
        price = client.get_ticker(symbol="NEOUSDT")
        
        if currentorder['status'] == 'PARTIALLY_FILLED':
            nums = nums - float(currentorder['executedQty'])
        
        if myorder['side'] == 'SELL' and float(myorder['price']) > float(price['askPrice']):
            ## cancel order and reorder
            print 'cancel sell and reorder!'
            result = client.cancel_order(
            symbol='NEOUSDT',
            orderId=myorder['orderId'])
            myorder = binanceorder(buyorsell, nums, client)
        elif myorder['side'] == 'BUY' and float(myorder['price']) < float(price['bidPrice']):
            print 'cancel buy and reorder!'
            result = client.cancel_order(
            symbol='NEOUSDT',
            orderId=myorder['orderId'])
            myorder = binanceorder(buyorsell, nums, client)
        time.sleep(5)
        currentorder = client.get_order(symbol='NEOUSDT', orderId=myorder['orderId'])

In [ ]:
# example:  binanceorder_fight('b', 0.1, client)